In [1]:
# Imports:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# Preprocessing

# Load the CSV file into a DataFrame
df = pd.read_csv('dataset.csv')

# Define the genres you are interested in
genre_filter = ['acoustic', 'electronic', 'pop', 'rock', 'folk', 'emo', 'alternative']

# Filter the DataFrame to only include rows where the Genre column matches one of the desired genres
filtered_df = df[df['track_genre'].isin(genre_filter)]

# Display the filtered DataFrame
# print(filtered_df)

# Optional: Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('filtered_dataset.csv', index=False)

# Calculate the distribution of the 'track_genre' column
genre_distribution = filtered_df['track_genre'].value_counts()

# Print the distribution
print(genre_distribution)

       Unnamed: 0                track_id                          artists  \
0               0  5SuOikwiRyPMVoIQDJUgSV                      Gen Hoshino   
1               1  4qPNDBW1i3p13qLCt0Ki3A                     Ben Woodward   
2               2  1iJBSr7s7jYXzM8EGcbK5b           Ingrid Michaelson;ZAYN   
3               3  6lfxq3CG4xtTiEg7opyCyx                     Kina Grannis   
4               4  5vjLSffimiIP26QG5WcN2K                 Chord Overstreet   
...           ...                     ...                              ...   
91995       91995  5J5VBW8G9ViJfzX5FEYvni  Tom Petty and the Heartbreakers   
91996       91996  5nQqnfHX5cr2FsGszRtTpr  Tom Petty and the Heartbreakers   
91997       91997  0sL9GKpJKZRngHncF2r0Rg  Tom Petty and the Heartbreakers   
91998       91998  6S70OrI0CsYO0nD1l6qvDP  Tom Petty and the Heartbreakers   
91999       91999  75MWtEDkqlSoJAXrdw1lR9  Tom Petty and the Heartbreakers   

                                              album_name  \
0  